In [81]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
from data.acronymDataset import AcronymDataset
from evaluate import load
import numpy as np

In [3]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=config.max_position_embeddings)
pre_trained_model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test with the data

In [66]:
%load_ext autoreload
%autoreload 2
file_path = 'data/acronym_data.txt'
dataset = AcronymDataset(file_path=file_path, tokenizer=tokenizer)
data = dataset.data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
dataset.preprocss_dataset()

In [69]:
%load_ext autoreload
%autoreload 2
train_loader, val_loader = dataset.get_dataloaders(train_size=0.9, batch_size=16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
for i, batch in enumerate(train_loader):
    print(batch["input_ids"].shape)

torch.Size([16, 246])
torch.Size([16, 329])
torch.Size([16, 274])
torch.Size([16, 291])
torch.Size([16, 474])
torch.Size([16, 319])
torch.Size([16, 269])
torch.Size([16, 274])
torch.Size([16, 310])
torch.Size([16, 413])
torch.Size([16, 333])
torch.Size([16, 283])
torch.Size([16, 323])
torch.Size([16, 339])
torch.Size([16, 354])
torch.Size([16, 356])
torch.Size([16, 512])
torch.Size([16, 257])
torch.Size([16, 262])
torch.Size([16, 270])
torch.Size([16, 282])
torch.Size([16, 276])
torch.Size([16, 270])
torch.Size([16, 414])
torch.Size([16, 420])
torch.Size([16, 274])
torch.Size([16, 257])
torch.Size([16, 302])
torch.Size([16, 334])
torch.Size([16, 286])
torch.Size([16, 260])
torch.Size([16, 282])
torch.Size([16, 312])
torch.Size([16, 298])
torch.Size([16, 297])
torch.Size([16, 267])
torch.Size([16, 351])
torch.Size([16, 279])
torch.Size([16, 368])
torch.Size([16, 346])
torch.Size([16, 267])
torch.Size([16, 312])
torch.Size([16, 512])
torch.Size([16, 304])
torch.Size([16, 260])
torch.Size

KeyboardInterrupt: 

In [72]:
batch = next(iter(train_loader))

In [39]:
from models.multiHeadModel import MultiHeadModel
from models.heads import ClassificationHead

In [73]:
in_features = config.hidden_size
two_labels_head = ClassificationHead(in_features=in_features, out_features=2)
four_labels_head = ClassificationHead(in_features=in_features, out_features=4)

classifiers = [two_labels_head, four_labels_head]


In [74]:
multi_head_model = MultiHeadModel(pre_trained_model, classifiers)

In [75]:

output = multi_head_model(batch, 0)
output

tensor([[0.5152, 0.4353],
        [0.5096, 0.4345],
        [0.5118, 0.4289],
        [0.5214, 0.4492],
        [0.5024, 0.4435],
        [0.5210, 0.4184],
        [0.5176, 0.4250],
        [0.4991, 0.4375],
        [0.5063, 0.4391],
        [0.5033, 0.4364],
        [0.5077, 0.4114],
        [0.5084, 0.4114],
        [0.5083, 0.4325],
        [0.5055, 0.4319],
        [0.5112, 0.4492],
        [0.5009, 0.4470]], grad_fn=<SigmoidBackward0>)

In [83]:
metric = load("accuracy")
labels = batch['labels']
predictions = np.argmax(output.detach().numpy(), axis=-1)
res = metric.compute(predictions=predictions, references=labels)

res

{'accuracy': 0.25}

In [9]:
output = multi_head_model(batch, 1)
output

tensor([[0.5332, 0.4287, 0.5225, 0.5521]], grad_fn=<SigmoidBackward0>)